In [3]:
import os
from crate import client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import random
from agots.multivariate_generators.multivariate_data_generator import MultivariateDataGenerator

In [8]:
connection = client.connect("http://localhost:4200/", username="crate")

In [9]:
cursor= connection.cursor()

In [11]:
from sqlalchemy import create_engine
engine = create_engine('crate://localhost:4200')

In [27]:
#file=pd.read_csv(r'data.csv', sep=',', header=0, low_memory=False)
#file.drop('timestamps', axis='columns', inplace=True)
#date_range=pd.date_range(start='9/4/1985', periods=len(file), freq='D')
#file['timestamps']=pd.to_datetime(date_range)
#file.index=file['timestamps']
#anomalies=file.loc[file['anomaly']==True]
#anomalies=anomalies[['value','anomaly']]
#file=file[['value']]
#anomalies.head()

In [26]:
#cursor.execute("DELETE FROM time")
#file.to_sql('time', con=engine, if_exists='append', chunksize=1000000)
#cursor.execute("DELETE FROM anomalies")
#anomalies.to_sql('anomalies', con=engine, if_exists = 'append', chunksize=100000)

In [2]:
end_date='9/3/2020'

date_rng = pd.date_range(start='9/4/1985', end=end_date, freq='D')

In [3]:
STREAM_LENGTH = len(date_rng)
N = 1
K = 1

dg = MultivariateDataGenerator(STREAM_LENGTH, N, K)
df = dg.generate_baseline(initial_value_min=-2, initial_value_max=2)

In [15]:
anomaly=[]
for i in range(random.randrange(10)):
    x=9862+random.randrange(2922)
    dg.add_outliers({'extreme': [{'n': 0, 'timestamps': [(x,)],'factor': 10}]})
    anomaly.append(x)
anomaly.sort()
anomaly=np.array(anomaly)
anomaly=pd.to_datetime(anomaly,origin='9/4/1985', unit='D')
anomalies=pd.DataFrame(anomaly, columns=['timestamps'])
anomalies['anomaly']=True
anomalies.index=anomalies['timestamps']
anomalies=anomalies[['anomaly']]

In [5]:
df['date']=date_rng
df['timestamps'] = pd.to_datetime(df['date'])
df.drop(['date'], axis=1, inplace=True)
df.index=df['timestamps']
df.columns=['value', 'timestamps']
df=df[['value']]
df.head(10)

,value
timestamps,
1985-09-04,0.000000
1985-09-05,1.026622
1985-09-06,1.022128
1985-09-07,1.519825
1985-09-08,1.128708
1985-09-09,1.308359
1985-09-10,1.971159
1985-09-11,2.128206
1985-09-12,0.873162


In [14]:
cursor.execute("DELETE FROM time")
df.to_sql('time', con = engine, if_exists = 'append', chunksize = 100000)

In [50]:
cursor.execute("DELETE FROM anomalies")
anomalies.to_sql('anomalies', con=engine, if_exists = 'append', chunksize=100000)